# Limpieza de datos – Ventas

Objetivos del notebook:
- Dataset sin duplicados ni nulos.
- Columnas estandarizadas y consistentes (snake_case).
- Código claro y comentado.
- Observaciones finales con métricas de limpieza.

In [6]:
# Importaciones y rutas
import pandas as pd
import unicodedata as ud
from pathlib import Path

DATA_PATH = Path('..') / 'data' / 'sales_data_sample.csv'
OUTPUT_PATH = Path('..') / 'output' / 'sales_data_clean.csv'

## 1. Carga y exploración inicial

In [7]:
# Cargar datos crudos
df = pd.read_csv(DATA_PATH, encoding='latin1')
rows_raw, cols_raw = df.shape
print(f'Filas crudas: {rows_raw} | Columnas crudas: {cols_raw}')
df.head()

Filas crudas: 2823 | Columnas crudas: 25


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [8]:
# Resumen de tipos y nulos por columna
print('Info:')
df.info()
print('Nulos por columna:')
nulls_before = df.isna().sum().sort_values(ascending=False)
nulls_before

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    ob

ADDRESSLINE2        2521
STATE               1486
TERRITORY           1074
POSTALCODE            76
SALES                  0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
ORDERNUMBER            0
MONTH_ID               0
QTR_ID                 0
STATUS                 0
ORDERDATE              0
PRODUCTCODE            0
YEAR_ID                0
MSRP                   0
PRODUCTLINE            0
ADDRESSLINE1           0
PHONE                  0
CUSTOMERNAME           0
CITY                   0
COUNTRY                0
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
dtype: int64

## 2. Plan de limpieza
- Estandarizar nombres de columnas (minúsculas, snake_case, sin tildes).
- Eliminar duplicados.
- Tipificar columnas clave (por ejemplo, fechas).
- Imputar nulos en campos categóricos frecuentes (por ejemplo, 'state', 'territory') con 'Unknown'.
- Imputar nulos en campos opcionales de texto con cadena vacía (por ejemplo, 'addressline2', 'postalcode').

In [9]:
# Estandarizar nombres de columnas
def normalize_col(name: str) -> str:
    name = name.strip().lower()
    # eliminar tildes/acentos
    name = ud.normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
    # reemplazar espacios y guiones por '_' y quitar caracteres no alfanum/underscore
    name = name.replace('-', ' ').replace('/', ' ')
    name = '_'.join(name.split())
    name = ''.join(ch for ch in name if ch.isalnum() or ch == '_')
    return name

df.columns = [normalize_col(c) for c in df.columns]
print('Columnas estandarizadas:')
print(df.columns.tolist())

Columnas estandarizadas:
['ordernumber', 'quantityordered', 'priceeach', 'orderlinenumber', 'sales', 'orderdate', 'status', 'qtr_id', 'month_id', 'year_id', 'productline', 'msrp', 'productcode', 'customername', 'phone', 'addressline1', 'addressline2', 'city', 'state', 'postalcode', 'country', 'territory', 'contactlastname', 'contactfirstname', 'dealsize']


In [10]:
# Tipificar columnas clave (fechas, códigos)
if 'orderdate' in df.columns:
    # parseo a datetime sin forzar NaT
    df['orderdate'] = pd.to_datetime(df['orderdate'])
# asegurar que postalcode sea texto (preserva ceros a la izquierda)
if 'postalcode' in df.columns:
    df['postalcode'] = df['postalcode'].astype('string')
# limpiar espacios en textos
for c in df.select_dtypes(include=['object', 'string']).columns:
    df[c] = df[c].astype('string').str.strip()

In [11]:
# Eliminar duplicados
rows_before_dupes = len(df)
df = df.drop_duplicates().reset_index(drop=True)
dupes_removed = rows_before_dupes - len(df)
print(f'Duplicados eliminados: {dupes_removed}')

Duplicados eliminados: 0


In [12]:
# Imputación controlada de nulos para asegurar dataset sin nulos
fill_map = {}
for col in ['addressline2', 'postalcode']:
    if col in df.columns:
        fill_map[col] = ''
for col in ['state', 'territory']:
    if col in df.columns:
        fill_map[col] = 'Unknown'
# aplicar imputación específica
if fill_map:
    df = df.fillna(value=fill_map)
# si quedaran nulos residuales: rellenar textos con '' y numéricos con 0
obj_cols = df.select_dtypes(include=['object', 'string']).columns
num_cols = df.select_dtypes(include=['number']).columns
dt_cols = df.select_dtypes(include=['datetime']).columns
df[obj_cols] = df[obj_cols].fillna('')
df[num_cols] = df[num_cols].fillna(0)
for c in dt_cols:
    if df[c].isna().any():
        # usar la moda de la fecha si existe, si no, epoca por defecto
        mode_val = df[c].mode().iloc[0] if not df[c].mode().empty else pd.Timestamp('1970-01-01')
        df[c] = df[c].fillna(mode_val)

print('Nulos totales tras imputación:', int(df.isna().sum().sum()))

Nulos totales tras imputación: 0


In [13]:
# Validación: no nulos y columnas consistentes
nulls_after = df.isna().sum()
print('Nulos por columna (después):')
print(nulls_after[nulls_after > 0])
assert df.isna().sum().sum() == 0, 'Aún quedan valores nulos'
print('OK: Dataset sin nulos.')

Nulos por columna (después):
Series([], dtype: int64)
OK: Dataset sin nulos.


## 3. Observaciones
- Se estandarizaron nombres de columnas a `snake_case` sin tildes.
- Se eliminaron filas duplicadas.
- `orderdate` se parseó a tipo fecha.
- Se imputaron nulos:
  - Categóricos como `state` y `territory` con 'Unknown'.
  - Textos opcionales como `addressline2` y `postalcode` con cadena vacía.
- Se validó que no queden nulos en el dataset.

In [15]:
# Métricas de limpieza
rows_clean, cols_clean = df.shape
print(f'Filas finales: {rows_clean} | Columnas finales: {cols_clean}')
print(f'Duplicados eliminados: {dupes_removed}')
print('Top columnas con nulos (antes):')
print(nulls_before.head(10))
print('Comprobación final de nulos totales:', int(df.isna().sum().sum()))

Filas finales: 2823 | Columnas finales: 25
Duplicados eliminados: 0
Top columnas con nulos (antes):
ADDRESSLINE2       2521
STATE              1486
TERRITORY          1074
POSTALCODE           76
SALES                 0
QUANTITYORDERED       0
PRICEEACH             0
ORDERLINENUMBER       0
ORDERNUMBER           0
MONTH_ID              0
dtype: int64
Comprobación final de nulos totales: 0


In [16]:
# Exportar dataset limpio
df.to_csv(OUTPUT_PATH, index=False)
print(f'Dataset limpio guardado en: {OUTPUT_PATH}')

Dataset limpio guardado en: ..\output\sales_data_clean.csv
